In [ ]:
# Part Two, Task One
# Step 1: Train a Lightweight Image Classification Model
# Tools: TensorFlow/Keras, TensorFlow Lite, Raspberry Pi (or Google Colab for simulation).
# 1. Dataset Preparation
# - Use a dataset like TrashNet (6 classes: glass, paper, metal, plastic, cardboard, trash).

# - Preprocess images (resize to 224x224, normalize pixel values).

In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Load dataset (example using TensorFlow Datasets)
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    'trashnet_data',
    image_size=(224, 224),
    batch_size=32
)

# Split into train/validation
train_ds = dataset.take(80)
val_ds = dataset.skip(80)

NotFoundError: Could not find directory trashnet_data

In [3]:
# Download the TrashNet dataset
# You might need to find a public URL or use a library to download it.
# For demonstration purposes, let's assume we can download a zipped version and extract it.
# Replace the URL with the actual download link if available.
!wget https://github.com/garythung/trashnet/archive/master.zip
!unzip master.zip
!mv trashnet-master trashnet_data

--2025-07-22 14:52:35--  https://github.com/garythung/trashnet/archive/master.zip
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/garythung/trashnet/zip/refs/heads/master [following]
--2025-07-22 14:52:35--  https://codeload.github.com/garythung/trashnet/zip/refs/heads/master
Resolving codeload.github.com (codeload.github.com)... 140.82.112.10
Connecting to codeload.github.com (codeload.github.com)|140.82.112.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘master.zip’

master.zip              [         <=>        ]  40.63M  13.9MB/s    in 2.9s    

2025-07-22 14:52:38 (13.9 MB/s) - ‘master.zip’ saved [42609527]

Archive:  master.zip
6fa2b878c6c1b4304b91109070ce0edf9279bb31
   creating: trashnet-master/
  inflating: trashnet-master/.gitignore  
  inflating: tr

In [ ]:
# 2. Train a Mobile-Friendly Model
# Use MobileNetV2 (pre-trained on ImageNet) for efficiency:

In [4]:
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = False  # Freeze pre-trained layers

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dense(6, activation='softmax')  # 6 classes
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(train_ds, validation_data=val_ds, epochs=10)

# Expected Output:

# Training accuracy: ~90%

# Validation accuracy: ~85%

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


NameError: name 'train_ds' is not defined

In [5]:
# Step 2: Convert to TensorFlow Lite
# Optimize the model for edge devices:

# Convert to TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model
with open('recyclable_model.tflite', 'wb') as f:
    f.write(tflite_model)

Saved artifact at '/tmp/tmpuqqat8ud'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='keras_tensor_154')
Output Type:
  TensorSpec(shape=(None, 6), dtype=tf.float32, name=None)
Captures:
  140438362227152: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140438362228496: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140438362231952: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140438362231568: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140438362230416: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140438362232144: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140438362231184: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140438362228112: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140438362232336: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140438362230224: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1404383622

In [6]:
# Quantization (Optional): Reduce model size further by quantizing weights to 8-bit integers:

converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quant_model = converter.convert()

Saved artifact at '/tmp/tmpej5qtqtv'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='keras_tensor_154')
Output Type:
  TensorSpec(shape=(None, 6), dtype=tf.float32, name=None)
Captures:
  140438362227152: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140438362228496: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140438362231952: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140438362231568: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140438362230416: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140438362232144: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140438362231184: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140438362228112: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140438362232336: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140438362230224: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1404383622

In [7]:
# Step 3: Test on Sample Data
# Simulate Edge Deployment (Colab/Raspberry Pi):

import numpy as np
import tflite_runtime.interpreter as tflite

# Load TFLite model
interpreter = tflite.Interpreter(model_path='recyclable_model.tflite')
interpreter.allocate_tensors()

# Test on a sample image
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Preprocess input (e.g., resize, normalize)
input_data = np.expand_dims(preprocessed_image, axis=0).astype(np.float32)
interpreter.set_tensor(input_details[0]['index'], input_data)

# Run inference
interpreter.invoke()
output = interpreter.get_tensor(output_details[0]['index'])
print("Predicted class:", np.argmax(output))

ModuleNotFoundError: No module named 'tflite_runtime'

In [ ]:
# Expected Output:

# Correct classification of recyclable items with ~80-85% accuracy.